Data : https://github.com/songys/Chatbot_data


In [1]:
! pip install transformers
! pip install pytorch-lightning
! pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 36.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
U_TKN = '<usr>'
S_TKN = '<sys>'
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

* U_TKN = "" : 질문 유저 토큰
* S_TKN = "" : 대답 시스템 토큰
* BOS = '' : 문장의 시작
* EOS = '' : 문장의 끝을
* MASK = '' : 마스크 처리
* SENT = '' : 문장 처리
* PAD = '' : 패딩

In [5]:
#모델 및 토크나이저 정의
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [6]:
# Setting parameters
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epoch = 70
Sneg = -1e18

In [7]:
#챗봇데이터셋
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=100):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = U_TKN
        self.a_token = S_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [8]:
Chatbot_Data=pd.read_csv('/content/drive/MyDrive/kogpt2_aicomment/chatbot_dataset_s.csv')
Chatbot_Data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [9]:
train_set = ChatbotDataset(Chatbot_Data, max_len=100)
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


In [17]:
import matplotlib.pyplot as plt

epoch=5

losses = []
X = []
Y = []
print("start")
for epoch in range(epoch):
    print(epoch)
    X.append(epoch)
    for batch_idx, samples in enumerate(train_dataloader): #train_dataloader에서 token_ids, mask, label 값을 할당
        optimizer.zero_grad() #optimizer를 초기화하고, 모델의 출력 값 out을 계산, logits 메서드를 이용하여 로짓 값을 계산
        token_ids, mask, label = samples
        model=model.to(device)
        token_ids = token_ids.to(device)
        mask = mask.to(device)
        label = label.to(device)
        out = model(token_ids)
        out = out.logits.to(device)

        #mask 값을 3차원으로 확장, out과 동일한 크기로 반복하여 mask_3d를 생성.
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2).to(device)
        #torch.where 메서드를 이용하여 mask가 1인 위치는 out 값을,
        #0인 위치는 Negative infinity 값을 가지는 텐서를 생성하여 mask_out 변수에 할당
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out)).to(device)
        loss = criterion(mask_out.transpose(2, 1), label).to(device)

        #손실 함수(criterion)를 계산하고, loss 값을 이용하여 현재 손실 값을 구함
        # 이때, 손실 값의 평균을 구하기 위해 avg_loss 변수를 계산
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward() #backward 메서드를 이용하여 그라디언트를 계산하고, step 메서드를 이용하여 파라미터를 업데이트
        optimizer.step()
    Y.append(loss.data.cpu().numpy())
print("end")

start
0
1
2
3
4
end


In [32]:
#모델 저장
torch.save(model, '/content/drive/MyDrive/kogpt2_aicomment/Kogpt2_epoch5.pth')

In [26]:
def kogpt(input_text): #koGPT2 모델을 활용하여 입력된 질문에 대한 대답을 생성하는 함수
    q = input_text
    a = ""
    sent = ""
    while True:
        input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(U_TKN + q + SENT + sent + S_TKN + a)).unsqueeze(dim=0)
        input_ids=input_ids.to(device)
        pred = model(input_ids) # 인코딩된 input_ids를 모델에 입력으로 넣어 pred 변수에 예측 결과를 저장
        pred = pred.logits
        #torch.argmax 함수를 이용하여 가장 확률이 높은 토큰을 선택하고, 이를 gen 변수에 저장,
        #이때 convert_ids_to_tokens 함수를 이용하여 선택된 토큰을 문자열로 변환
        gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().tolist())[-1]
        if gen == EOS: #gen 변수가 EOS인 경우, 생성 과정을 마침. 그렇지 않은 경우 gen에 대한 값을 a에 추가
            break
        a += gen.replace("▁", " ")
    return a

In [31]:
input_text = "친한 친구와 심하게 싸워서 너무 우울해"
kogpt(input_text)

' 서로 오해가 쌓여서 감정의 골이 깊어지는 것 같아요  많이 답답하시겠어요 '

In [28]:
diary='''
물리적으로도 피곤하고 마음 속으로도 지친 하루였다. 하루종일 뜬눈으로 시간을 보냈다.
아침 일어나자마자부터 마음이 무겁고 뭔가 잘못된 기분이 들었다.
머릿속이 복잡해서 무엇부터 해야 할지 막막했다. 친구들과의 대화조차도 어색하게 느껴졌다.
하루 종일 날이 어두워져가는 것처럼 마음도 점점 어두워져만 갔다.
이럴 때마다 왜 이렇게 우울해지는 건지 모르겠다. 아무것도 재미있게 느껴지지 않았다.
음악도, 영화도, 책도, 아무것도 마음에 들지 않았다.
책상 위에는 무언가를 해야겠다는 생각으로 쌓아둔 책과 종이들이 더럽혀져만 가는데, 손을 대기 싫었다.
'''
kogpt(diary)

' 작은 성취감을 느끼는 게 우울함을 없애줄지도 몰라요 '